In [1]:
import torch
import torch.nn as nn
import fastai.vision.all as fv

In [2]:
torch.cuda.is_available()

True

Cuando clasificamos aves lo que nos interesa es maximizar la accuracy (minimizar la taza de error)

No es buena idea maximizarla directamente, pues sólo se fija en el máximo y deja todo lo demás. Tampoco es continua. 

Dice si la imagen es o no es. Entonces debemos maximizar otra función de pérdida.

In [3]:
def load_data(folder, img_size,batch_size):
    tmfs = fv.aug_transforms(flip_vert=False,
                            max_rotate=10,
                            max_lighting=0.25,
                            max_zoom=1.2,
                            max_warp=0.2)
    data = fv.DataBlock(blocks  = (fv.ImageBlock, fv.CategoryBlock),
                        get_items = fv.get_image_files,
                        get_y     = fv.parent_label, #Como a partir de la imagen encpontarr la categoria
                        splitter  = fv.GrandparentSplitter(), #si el abyelo es train o test
                        item_tfms = fv.Resize(img_size),#transformcaiones antes de juntar en batch y despues de juntar en batch)
                        batch_tfms = tmfs)
    return data.dataloaders(folder, bs=batch_size)#Agarra las imagenes y las regresa en batches

birds = load_data("/storage/birds", img_size=128, batch_size=64)

In [4]:
##Flatten es una capa que ayuda a pasar de la parte convolucional a la parte lineal

#Lo que hace la clase es que le pasamos algo y nos regresa eso mismo pero sin los 1
class Flatten(nn.Module):   #Creamos un módulo que hereda de nn.Module
    def __init__(self):    #Para crear un objeto
        super().__init__() #Le decimos a nn.Module que se inicie
    
    def forward( self, x ): #
        #return x.squeeze()
        return x.reshape (x.shape[0],-1) #Toma la primera y adivina el resto
        #Se agrega esta porque si el batch es de tam 1, squezze lo elimina

lx97  = nn.Sequential( #Sequential para cosas sencillas
    nn.BatchNorm2d( 3 ), #Ideal comenzar normalizando 
    #Convolucion de entrada 3, de 32 filtros, de tamaño 3x3, 
    #sride -> ¿cuánto brinca?, padding -> pone el brde negro para no perder pixeles
    nn.Conv2d( 3, 32, kernel_size = 3, stride = 2, padding = 1),
    
    #Acivacion
    nn.ReLU(),
    ##ResBlock( crear_residual( 16 ) ),
    #Reducir el tamaño de a  imagen (será más rápido)
    #Cada cuadro de 2x2 agarra el más grande.
    #Reduce a la mitad
    #nn.MaxPool2d(2),
    #nn.Conv2d( 16, 32, kernel_size=2 ),
    #nn.ReLU(),
    nn.BatchNorm2d( 32 ),
    nn.Conv2d( 32, 32, kernel_size=2, stride = 1, padding = 1 ),
    nn.ReLU(),
    nn.BatchNorm2d( 32 ),
    
    #Empezamos en la cantidad de filtros con los que acabamos en Conv2d anterior
    nn.Conv2d( 32, 64, kernel_size = 2, stride = 1, padding = 1),
    
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d( 64, 128, kernel_size = 3, stride = 1, padding = 1),
    nn.ReLU(),
    nn.BatchNorm2d( 128 ),
    nn.MaxPool2d(2),
    nn.Conv2d( 128, 256, kernel_size = 3, stride = 1, padding = 1),
    nn.BatchNorm2d( 256 ),
    nn.ReLU(),
    nn.Conv2d( 256, 256, kernel_size = 3, stride = 1, padding = 1),
    nn.BatchNorm2d( 256 ),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d( 256, 512, kernel_size = 3, stride = 1, padding = 1),
    nn.BatchNorm2d( 512 ),
    nn.ReLU(),
    #--Aquí acabamos con la parte convolucional--#
    
    #--Llegamos a la parte lineal--#
    
    nn.AdaptiveAvgPool2d(1), #Reducir el tamaño a 1x1xn_canales
    #Podemos usar diferentes métodos para llegar a 1x1
    Flatten(), #Usamos la funcion que habiamos creado pra eliminar las de dim 1
    #Los quita para poder usar Linear
    nn.Linear( 512, 256), #Completamente conectada
    nn.ReLU(),
    nn.Linear( 256, birds.c) #salida con el total de clases de los datos
)

In [5]:
x,y = birds.one_batch()

In [6]:
x.shape

torch.Size([64, 3, 128, 128])

In [7]:
y,y.shape

(TensorCategory([170, 145,  72, 170,  46,  74,  14,  62, 157,  57,   0,  14,  25, 100,
          66,  40,  36, 101,   9,   1,  78,  97, 159, 139,  36,  23,  80,  86,
         169,  98, 126,  11,  73, 188,  21, 129, 179, 172,  72, 135,  64, 123,
          55,  41, 185, 185, 133, 181,   4, 100,  22, 150, 150, 124,  88, 109,
         186, 142, 198, 125, 107,  77, 184,  78], device='cuda:0'),
 torch.Size([64]))

x -> batch de imágenes
y -> Tensor de números enteros

## Cross Entropy

Función para clasificar.
Maximiza la probabilidad de clasificar correctamente

Le debemos pasar yp y y sin softmax y sin one-hot-encodear